In [1]:
import subprocess

import requests
from pystac_client import Client

In [2]:
catalog = Client.open("http://ec2-54-172-212-55.compute-1.amazonaws.com/api/v1/pgstac/")

# Washington, DC
bbox = [-76.964657, 38.978967, -76.928008, 39.002783]
search = catalog.search(
    collections=["landsat-c2l1"],
    bbox=bbox,
)

items = search.get_all_items()
item = items[0]

In [3]:
item_cid = item.assets["red"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]
item_cid

'QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To'

In [5]:
def check_local_ipfs():
    try:
        ipfs_present = subprocess.check_output(["ipfs", "version"]).decode("utf-8")
        print(f"Local IPFS Node Detected! {ipfs_present}")
        return True
    except subprocess.CalledProcessError as e:
        print(e.output.decode("utf-8"))
        return False


def get_data_from_local_ipfs(cid):
    try:
        subprocess.check_output(["ipfs", "pin", "ls", cid])
        print(f"CID: {cid} found in local IPFS node")
        data = subprocess.check_output(["ipfs", "cat", cid])
        return data
    except subprocess.CalledProcessError as e:
        print(e.output.decode("utf-8"))
        return None


def fetch_from_ipfs_network(cid):
    try:
        print("Attempting to fetch CID from IPFS network using local IPFS node")
        data = subprocess.check_output(["ipfs", "cat", cid])
        print(f"CID: {cid} found on IPFS network via local IPFS node")
        print("Pinning CID to local IPFS node")
        print(subprocess.check_output(["ipfs", "pin", "add", cid]))
        return data
    except subprocess.CalledProcessError as e:
        print(e.output.decode("utf-8"))
        return None


def fetch_from_ipfs_gateway(cid):
    try:
        data = requests.get(f"https://ipfs.io/ipfs/{cid}").content
        print(f"CID: {cid} found on IPFS network via Gateway")
        return data
    except requests.exceptions.RequestException as e:
        print(e.output.decode("utf-8"))
        return None


def get_data(cid):
    local_ipfs_present = check_local_ipfs()

    if local_ipfs_present:
        data = get_data_from_local_ipfs(cid)
        if data is None:
            data = fetch_from_ipfs_network(cid)

    else:
        data = fetch_from_ipfs_gateway(cid)

    return data


data = get_data(item_cid)

Local IPFS Node Detected! ipfs version 0.18.1

CID: QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To found in local IPFS node


 82.62 MiB / 82.62 MiB  100.00% 0s
